In [1]:
import cv2
import numpy as np

# 读取视频文件
cap = cv2.VideoCapture('降采样的标记点增强序列.mp4')

# 初始化保存两个黑点的列表
black_points = []

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # 对帧进行预处理（二值化、边缘检测等）
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    edges = cv2.Canny(binary, 50, 150)
    
    # 查找轮廓
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 清空黑点列表
    black_points.clear()
    
    # 选择符合条件的黑点
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 15:  # 根据黑点的面积进行筛选
            M = cv2.moments(contour)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            black_points.append((cx, cy))
    
    # 绘制黑点
    for point in black_points:
        cv2.circle(frame, point, 5, (0, 0, 255), -1)
    
    # 计算斜率
    if len(black_points) == 2:  # 只有两个黑点时计算斜率
        x1, y1 = black_points[0]
        x2, y2 = black_points[1]
        slope = (y2 - y1) / (x2 - x1 +1e-20)
        cv2.putText(frame, f"Slope: {slope:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # 显示结果
    cv2.imshow('Frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:

import cv2
import numpy as np
import csv

# 读取视频文件
cap = cv2.VideoCapture('降采样的液体晃动增强.mp4')

# 初始化斜率列表
slopes = []

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # 将帧转换为灰度图像
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 对图像进行边缘检测
    edges = cv2.Canny(gray, 5, 200)
    
    # 进行霍夫直线检测
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=150, minLineLength=100, maxLineGap=100)
    
    # 找到最长的直线
    max_length = 0
    longest_line = None
    if lines is not None:  # 添加判断以确保至少检测到一条直线
        for line in lines:
            x1, y1, x2, y2 = line[0]
            length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            if length > max_length:
                max_length = length
                longest_line = line
    
    # 绘制最长的直线
    if longest_line is not None:
        x1, y1, x2, y2 = longest_line[0]  # 修改此处以解决ValueError
        cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # 计算斜率
        slope = (y2 - y1) / (x2 - x1 + 1e-6)  # 避免除以零错误
        
        # 将斜率添加到列表中
        slopes.append(slope)
    
        cv2.putText(frame, f"Slope: {slope:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # 显示结果
    cv2.imshow('Frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 打印每一帧的斜率
for i, slope in enumerate(slopes):
    print(f"Frame {i+1}: Slope = {slope}")

# 定义CSV文件路径和文件名
csv_file = 'slopes.csv'

# 使用csv模块打开CSV文件并写入数据
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # 写入表头
    writer.writerow(['Slope'])
    
    # 写入每个斜率值
    for slope in slopes:
        writer.writerow([slope])

print("斜率信息已成功保存到CSV文件中。")

Frame 1: Slope = 0.03296703287646419
Frame 2: Slope = 0.016620498568918285
Frame 3: Slope = 0.033057851148600964
Frame 4: Slope = 0.03611111101080247
Frame 5: Slope = 0.0
Frame 6: Slope = 0.017391304297416508
Frame 7: Slope = 0.01666666662037037
Frame 8: Slope = 0.019498607188026165
Frame 9: Slope = 0.0
Frame 10: Slope = 0.03611111101080247
Frame 11: Slope = 0.0
Frame 12: Slope = 0.0
Frame 13: Slope = 0.0
Frame 14: Slope = 0.0
Frame 15: Slope = 0.019230769177937448
Frame 16: Slope = 0.01666666662037037
Frame 17: Slope = 0.016393442578159993
Frame 18: Slope = 0.017441860414413198
Frame 19: Slope = 0.0
Frame 20: Slope = 0.0
Frame 21: Slope = 0.036516853830008834
Frame 22: Slope = 0.01675977648949783
Frame 23: Slope = 0.0
Frame 24: Slope = 0.018018017963909855
Frame 25: Slope = 0.016528925574300482
Frame 26: Slope = 0.016483516438232097
Frame 27: Slope = 0.016713091875451
Frame 28: Slope = 0.019498607188026165
Frame 29: Slope = 0.016713091875451
Frame 30: Slope = 0.01675977648949783
Frame

In [1]:
import cv2
import numpy as np

# 读取视频文件
cap = cv2.VideoCapture('裁剪的原始视频.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # 对帧进行预处理（颜色过滤、灰度化等）
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 定义要追踪的颜色范围（这里以蓝色为例）
    lower_red = np.array([90, 50, 50])
    upper_red = np.array([130, 255, 255])
    
    # 根据颜色范围创建掩码
    mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # 进行形态学操作，填充小的空洞
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # 使用Hough圆变换检测圆形
    circles = cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=50, param2=30, minRadius=50, maxRadius=100)
    
    # 如果检测到圆形，则绘制圆形轮廓
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        
        for (x, y, r) in circles:
            cv2.circle(frame, (x, y), r, (0, 255, 0), 4)
            cv2.rectangle(frame, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
    
    # 显示结果
    cv2.imshow('Frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 